# FPsim Intervention Scenarios

With FPsim's user-friendly Scenarios API, we can easily setup and compare multiple intervention scenarios against one another to better understand the complex interactions between demographics and family planning behavior.

1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Efficacy](#efficacy)
4. [Switching](#switching)
5. [Initiation/Discontinuation](#initiation)
6. [Parameter changes](#parameter)
7. [Custom interventions](#custom)
8. [Comparison](#comparison)

## Introduction <a name="introduction"></a>

FPsim is a useful tool for understanding how demographics, contraceptive efficacy, and contraceptive uptake behaviors interact with one another. We will be going through a couple of scenarios using our "Scenarios" api, which makes it easier for researchers to customize and compare the results of family planning interventions.

## Setup <a name="setup"></a>
#### A simple guide on how to set up a basic intervention scenario

The below steps will demonstrate how we can model an increase in modern contraceptive efficacy and visualize the results.

In [ ]:
import sciris as sc
import fpsim as fp

In [ ]:
# First we create a Scenario object where:
#    eff is the dictionary of specifications for how we want to change efficacy
#    year is the year we want to implement the change
#    label is how we want to label the scenario in the visualization
scenario = fp.make_scen(eff={'Other modern':0.994}, year=2005, label='Increased modern')

# Then we add it to the Scenarios object which is used to compare results across multiple scenarios
# but for this case we will just be visualizing the one. Repeats defines how many times to run the 
# simulation in order to determine uncertainty intervals.
scenarios = fp.Scenarios(scens=scenario, repeats=3)

# Now that we have the scenario specifications, we can run the simulations
scenarios.run()

# To examine the resulting demoraphic features of the scenario we can use the plot_scens() function
scenarios.plot_scens()

## Efficacy <a name="efficacy"></a>
#### How to to compare sim results across different contraceptive efficacies

Let's suppose we want to better understand how demographic features change at varying levels of efficacy of modern contraceptive methods. For this task we would create three specification dictionaries, add them to scenarios, and then run.

In [ ]:
# In order to compare different scenarios to one another we must:

# Define the different scenarios
modern_995 = fp.make_scen(eff={'Other modern':0.994}, year=2005, label='Modern high efficacy')

modern_75 = fp.make_scen(eff={'Other modern':0.75}, year=2005, label='Modern moderate efficacy')

modern_50 = fp.make_scen(eff={'Other modern':0.5}, year=2005, label='Modern low efficacy')

# Add to scenarios object
scens_eff = fp.Scenarios(n=1000, repeats=3, scens=[modern_995, modern_75, modern_50])

# Run scenarios
scens_eff.run()

# Compare results
scens_eff.plot_scens()

## Switching <a name="switching"></a>
#### How to compare sim results across different switching probabilities.

If we want to understand how the probability of switching from one contraceptive to another affects a population, we may want to sweep this probability and examine the results. Below is how we would do this using Scenarios:

In [ ]:
# Within our scenario we can adjust the probability of switching from one intervention (source) to another (dest).
# For the purpose of granular analysis we can even target specific ages, and postpartum status which often affects
# a women's contraceptive uptake behavior. In the below example we target women in the first six months postpartum (matrix).
high_switching = fp.make_scen(source='Other modern', dest='Injectables', year=2005, value=0.95, ages='18-20', matrix='pp1to6', label="high switching")
medium_switching = fp.make_scen(source='Other modern', dest='Injectables', year=2005, value=0.75, ages='18-20', matrix='pp1to6', label="medium switching")
low_switching = fp.make_scen(source='Other modern', dest='Injectables', year=2005, value=0.50, ages='18-20', matrix='pp1to6', label="low switching")

# Then we pass these scenarios into the Scenarios object as before
scens_switching = fp.Scenarios(n=100, repeats=3, scens=[high_switching, medium_switching, low_switching])
scens_switching.run()

scens_switching.plot_scens()

## Initiation and Discontinuation <a name="initiation"></a> 
#### How to examine  initiation and discontinutation rates
It is often important to examine the probability of beginning a contraceptive method or terminating it. The below steps will demonstrate how to compare varying iniation and discontinuation rates using Scenario keyword arguments:

In [ ]:
# We can set the initiation factor by using the "method" and "init_factor" args.
# One can also set the value explicitly with init_value
high_init = fp.make_scen(label="Low init", method="Injectables", init_factor=1.5, year=2005)
medium_init = fp.make_scen(label="Medium init", method="Injectables", init_factor=1.0, year=2005)
low_init = fp.make_scen(label="high init", method="Injectables", init_factor=0.5, year=2005)

scens_initiation = fp.Scenarios(location="test", n=1000, scens=[high_init, medium_init, low_init])
scens_initiation.run()
scens_initiation.plot_scens()

In [ ]:
# discont_factor and discont_value are used for setting discontinuation
high_dis = fp.make_scen(year=2005, label="high dis", method="Injectables", discont_factor=1.5)
medium_dis = fp.make_scen(year=2005, label="medium dis", method="Injectables", discont_factor=1.0)
low_dis = fp.make_scen(year=2005, label="low dis", method="Injectables", discont_factor=0.5)

scens_discont = fp.Scenarios(location="test", n=1000, scens=[low_init, medium_init, high_init])
scens_discont.run()
scens_discont.plot_scens()

### **Neat trick**
One great feature of Scenario objects is that you can add them together. So we could combine the above scenarios:

In [ ]:
new_scenario = high_init + low_dis
new_scenario2 = low_init + high_dis

scens_combined = fp.Scenarios(location="test", n=1000, scens=[new_scenario, new_scenario2])
scens_combined.run()
scens_combined.plot_scens()

## Parameter changes <a name="parameter"></a>
#### How to change any parameter in the sim at a given year
Anything from the probabilitity of having twins to the exposure factor can be adjusted easily through keyword arguments: 

In [ ]:
low_exposure = fp.make_scen(par='exposure_factor', years=2000, vals=0.5)
medium_exposure = fp.make_scen(par='exposure_factor', years=2000, vals=1.0)
high_exposure = fp.make_scen(par='exposure_factor', years=2000, vals=1.5)

scens_combined = fp.Scenarios(location="test", n=1000, scens=[low_exposure, medium_exposure, high_exposure])
scens_combined.run()
scens_combined.plot_scens()

Specifically you can change any parameter in **People.pars** or **Sim.people.pars**

## Custom interventions <a name="custom"></a>
#### How to define a function that changes something in the model as it runs
As a sim runs, it applies "interventions" at each step. This is how we've acomplished the tasks in the past few sections. If none of the existing interventions are sufficient, it is possible to define and run one's own intervention:

In [ ]:
# At each step, the sim itself is passed into the intervention. This allows
# the user to access and change sim.People.pars, or parameters of the sim itself

# To acomplish the "low exposure" scenario from before, we would say:
def update_sim(sim): 
    if sim.y == 2002.0:
        sim.people.pars['exposure_factor'] = 0.5
        
new_scen = fp.Scenarios(label="Low exposure", scens=fp.make_scen(interventions=update_sim))
new_scen.run()

new_scen.plot_scens()

**Warning** This is an advanced use of Scenarios and requires some knowledge of the inner workings of the Sim class

## Comparison <a name="comparison"></a>
#### How to compare sim results in different ways

The Scenarios object has a number of methods for plotting results across different scenarios. Besides plot_scens() we also have **plot_sims()** which shows us the results of each individual sim run, **plot_cpr()** which shows us some statistics on CPR, and **analyze_sims()** which logs the results of the run as a handy pandas DataFrame.

**plot_sims()**

Plot sims is useful for when we want to understand the specific results of each individual sim for a given scenario. Although a group of sims for a single scenario should all have the same parameters, a difference in seed can produce a significant effect.

In [ ]:
# Single scenario with 3 runs
scenarios.plot_sims()

**plot_cpr()**
If we are more interested in the relationship between contraceptive usage and age, then we can utilize this function to narrow our focus. Namely it offers three visualizations to understand the prevalence of contraceptive use:
1. **mCPR** - modern contraceptive users who are women between 15-49
2. **CPR** - all contraceptive users who are women between 15-49
3. **ACPR** - all contraceptive users who are women that are sexually active and not pregnant


In [ ]:
scenarios.plot_cpr()

**analyze_sims()**

Lastly, if we want to create our own custom visualization, analyze_sims() let's us log the data as a dictionary or dataframe.

In [ ]:
scenarios.analyze_sims()

In [ ]:
# Dataframe format let's use the built-in dataframe functions for an aggregate view of the results
df = scenarios.results.df
df.plot(kind='bar', x="scenario", y="popsize")

In [ ]:
# If we want some summary statistics on the results of the scenarios, we can view the results as a dictinary using .stats
scens_eff.results.stats

## Further Exploration
### For more info check out the [source code](https://github.com/amath-idm/fp_analyses) 